# Gauss-Seidel Iterative Method for Linear Systems

## Introduction

The **Gauss-Seidel method** is an iterative technique for solving systems of linear equations of the form:

$$\mathbf{A}\mathbf{x} = \mathbf{b}$$

where $\mathbf{A} \in \mathbb{R}^{n \times n}$ is the coefficient matrix, $\mathbf{x} \in \mathbb{R}^n$ is the solution vector, and $\mathbf{b} \in \mathbb{R}^n$ is the right-hand side vector.

## Mathematical Foundation

### Matrix Decomposition

The Gauss-Seidel method decomposes the matrix $\mathbf{A}$ as:

$$\mathbf{A} = \mathbf{L} + \mathbf{D} + \mathbf{U}$$

where:
- $\mathbf{L}$ is the strictly lower triangular part of $\mathbf{A}$
- $\mathbf{D}$ is the diagonal of $\mathbf{A}$
- $\mathbf{U}$ is the strictly upper triangular part of $\mathbf{A}$

### Iteration Formula

The system $(\mathbf{L} + \mathbf{D} + \mathbf{U})\mathbf{x} = \mathbf{b}$ is rearranged as:

$$(\mathbf{L} + \mathbf{D})\mathbf{x}^{(k+1)} = \mathbf{b} - \mathbf{U}\mathbf{x}^{(k)}$$

The key insight of Gauss-Seidel is that when computing $x_i^{(k+1)}$, we use the **already updated** values $x_1^{(k+1)}, x_2^{(k+1)}, \ldots, x_{i-1}^{(k+1)}$ from the current iteration.

For each component $i = 1, 2, \ldots, n$:

$$x_i^{(k+1)} = \frac{1}{a_{ii}} \left( b_i - \sum_{j=1}^{i-1} a_{ij} x_j^{(k+1)} - \sum_{j=i+1}^{n} a_{ij} x_j^{(k)} \right)$$

### Convergence Criteria

The Gauss-Seidel method converges if any of the following conditions hold:

1. **Strictly Diagonally Dominant**: $|a_{ii}| > \sum_{j \neq i} |a_{ij}|$ for all $i$

2. **Symmetric Positive Definite**: $\mathbf{A} = \mathbf{A}^T$ and $\mathbf{x}^T \mathbf{A} \mathbf{x} > 0$ for all $\mathbf{x} \neq \mathbf{0}$

3. **Spectral Radius**: $\rho(\mathbf{T}_{GS}) < 1$ where $\mathbf{T}_{GS} = -(\mathbf{L} + \mathbf{D})^{-1}\mathbf{U}$

### Comparison with Jacobi Method

Unlike the Jacobi method which uses only values from the previous iteration, Gauss-Seidel incorporates newly computed values immediately. This typically results in faster convergence.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import norm

# Set random seed for reproducibility
np.random.seed(42)

## Implementation

We implement the Gauss-Seidel method with convergence tracking to visualize the iterative improvement of the solution.

In [ ]:
def gauss_seidel(A, b, x0=None, tol=1e-10, max_iterations=1000):
    """
    Solve Ax = b using the Gauss-Seidel iterative method.
    
    Parameters:
    -----------
    A : ndarray
        Coefficient matrix (n x n)
    b : ndarray
        Right-hand side vector (n,)
    x0 : ndarray, optional
        Initial guess (default: zeros)
    tol : float
        Convergence tolerance
    max_iterations : int
        Maximum number of iterations
    
    Returns:
    --------
    x : ndarray
        Solution vector
    residuals : list
        Residual norm at each iteration
    errors : list
        Solution error norm at each iteration
    """
    n = len(b)
    x = np.zeros(n) if x0 is None else x0.copy()
    
    # Compute exact solution for error tracking
    x_exact = np.linalg.solve(A, b)
    
    residuals = []
    errors = []
    
    for k in range(max_iterations):
        x_old = x.copy()
        
        # Gauss-Seidel iteration
        for i in range(n):
            # Sum of a_ij * x_j for j < i (using new values)
            sum1 = np.dot(A[i, :i], x[:i])
            # Sum of a_ij * x_j for j > i (using old values)
            sum2 = np.dot(A[i, i+1:], x_old[i+1:])
            # Update x_i
            x[i] = (b[i] - sum1 - sum2) / A[i, i]
        
        # Compute residual and error
        residual = norm(b - A @ x)
        error = norm(x - x_exact)
        
        residuals.append(residual)
        errors.append(error)
        
        # Check convergence
        if residual < tol:
            print(f"Converged after {k+1} iterations")
            break
    else:
        print(f"Maximum iterations ({max_iterations}) reached")
    
    return x, residuals, errors

## Test Case: Diagonally Dominant System

We construct a strictly diagonally dominant matrix to ensure convergence. For a $5 \times 5$ system, we create:

$$a_{ii} = 10, \quad |a_{ij}| \leq 1 \text{ for } j \neq i$$

In [ ]:
# Create a diagonally dominant matrix
n = 5
A = np.random.rand(n, n)  # Random entries in [0, 1]
A = A + A.T  # Make symmetric
A = A + n * np.eye(n)  # Make diagonally dominant

# Create right-hand side
b = np.random.rand(n)

print("Matrix A:")
print(A)
print("\nVector b:")
print(b)

In [ ]:
# Verify diagonal dominance
def check_diagonal_dominance(A):
    n = A.shape[0]
    for i in range(n):
        diag = abs(A[i, i])
        off_diag = sum(abs(A[i, j]) for j in range(n) if j != i)
        if diag <= off_diag:
            return False
    return True

print(f"Matrix is strictly diagonally dominant: {check_diagonal_dominance(A)}")

In [ ]:
# Solve using Gauss-Seidel
x_gs, residuals, errors = gauss_seidel(A, b, tol=1e-12)

# Compare with direct solution
x_exact = np.linalg.solve(A, b)

print("\nGauss-Seidel solution:")
print(x_gs)
print("\nExact solution:")
print(x_exact)
print(f"\nFinal error: {norm(x_gs - x_exact):.2e}")

## Convergence Analysis

We visualize the convergence behavior by plotting:
1. Residual norm $\|\mathbf{b} - \mathbf{A}\mathbf{x}^{(k)}\|$ vs iteration
2. Solution error $\|\mathbf{x}^{(k)} - \mathbf{x}^*\|$ vs iteration

The linear decrease on a semi-log plot indicates geometric convergence.

In [ ]:
# Create convergence plots
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Residual plot
axes[0].semilogy(residuals, 'b-', linewidth=2, marker='o', markersize=4)
axes[0].set_xlabel('Iteration', fontsize=12)
axes[0].set_ylabel(r'$\|\mathbf{b} - \mathbf{A}\mathbf{x}^{(k)}\|$', fontsize=12)
axes[0].set_title('Residual Convergence', fontsize=14)
axes[0].grid(True, alpha=0.3)

# Error plot
axes[1].semilogy(errors, 'r-', linewidth=2, marker='s', markersize=4)
axes[1].set_xlabel('Iteration', fontsize=12)
axes[1].set_ylabel(r'$\|\mathbf{x}^{(k)} - \mathbf{x}^*\|$', fontsize=12)
axes[1].set_title('Solution Error Convergence', fontsize=14)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('plot.png', dpi=150, bbox_inches='tight')
plt.show()

print("Plot saved to 'plot.png'")

## Comparison: Gauss-Seidel vs Jacobi

We compare the convergence rates of Gauss-Seidel and Jacobi methods on the same system.

In [ ]:
def jacobi(A, b, x0=None, tol=1e-10, max_iterations=1000):
    """
    Solve Ax = b using the Jacobi iterative method.
    """
    n = len(b)
    x = np.zeros(n) if x0 is None else x0.copy()
    x_exact = np.linalg.solve(A, b)
    
    residuals = []
    errors = []
    
    for k in range(max_iterations):
        x_new = np.zeros(n)
        
        # Jacobi iteration (uses only old values)
        for i in range(n):
            sum_term = np.dot(A[i, :], x) - A[i, i] * x[i]
            x_new[i] = (b[i] - sum_term) / A[i, i]
        
        x = x_new
        residual = norm(b - A @ x)
        error = norm(x - x_exact)
        
        residuals.append(residual)
        errors.append(error)
        
        if residual < tol:
            break
    
    return x, residuals, errors

# Solve with both methods
x_jacobi, res_jacobi, err_jacobi = jacobi(A, b, tol=1e-12)

print(f"Gauss-Seidel iterations: {len(residuals)}")
print(f"Jacobi iterations: {len(res_jacobi)}")

In [ ]:
# Comparison plot
plt.figure(figsize=(10, 6))
plt.semilogy(residuals, 'b-', linewidth=2, label='Gauss-Seidel')
plt.semilogy(res_jacobi, 'r--', linewidth=2, label='Jacobi')
plt.xlabel('Iteration', fontsize=12)
plt.ylabel('Residual Norm', fontsize=12)
plt.title('Convergence Comparison: Gauss-Seidel vs Jacobi', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

print(f"\nGauss-Seidel converges ~{len(res_jacobi)/len(residuals):.1f}x faster than Jacobi")

## Large System Example

We demonstrate the method on a larger sparse system arising from discretization of a 1D Poisson equation:

$$-\frac{d^2 u}{dx^2} = f(x)$$

with Dirichlet boundary conditions. The finite difference discretization yields a tridiagonal system.

In [ ]:
# Create tridiagonal Poisson matrix
n_large = 50
A_large = np.diag(2 * np.ones(n_large)) - np.diag(np.ones(n_large-1), 1) - np.diag(np.ones(n_large-1), -1)

# Source term (e.g., f(x) = sin(pi*x))
h = 1 / (n_large + 1)
x_pts = np.linspace(h, 1-h, n_large)
b_large = h**2 * np.sin(np.pi * x_pts)

# Solve
x_sol, res_large, err_large = gauss_seidel(A_large, b_large, tol=1e-10)

print(f"System size: {n_large} x {n_large}")
print(f"Iterations to converge: {len(res_large)}")

## Conclusion

The Gauss-Seidel method is an efficient iterative solver for linear systems, particularly when:

1. The matrix is **diagonally dominant** or **symmetric positive definite**
2. The system is **large and sparse** (where direct methods are expensive)
3. Only **moderate accuracy** is needed

Key advantages:
- Memory efficient (no matrix factorization required)
- Simple to implement
- Faster convergence than Jacobi due to immediate use of updated values

The convergence rate depends on the spectral radius of the iteration matrix, which is influenced by the matrix structure and conditioning.